# video-translation-service（Colab + T4 GPU）一键安装运行

目标：从直链下载 `1.mp4` → 生成中文字幕 `1_zh.srt`（不启用润色）。

注意：首次运行会下载 Whisper + 翻译模型（可能几 GB），需要等待一段时间。

In [ ]:
#@title 0) 确认已启用 GPU（Runtime -> Change runtime type -> GPU -> T4）
!nvidia-smi -L

import sys
import torch

print("python:", sys.version)
print("torch:", torch.__version__)
print("cuda_available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("gpu:", torch.cuda.get_device_name(0))
else:
    raise RuntimeError("未检测到GPU：请在 Colab 切换到 GPU(T4) 运行时")


In [ ]:
#@title 1) git clone 项目
# TODO: 改成你的仓库地址（私有仓库可用：https://<TOKEN>@github.com/<org>/<repo>.git）
REPO_URL = "https://github.com/MAE5blog/video-translation-service.git"
BRANCH = "main"
REPO_DIR = "/content/video-translation-service"

import os
import shutil

if os.path.exists(REPO_DIR):
    shutil.rmtree(REPO_DIR)

!git clone --depth 1 -b {BRANCH} {REPO_URL} {REPO_DIR}
%cd {REPO_DIR}


In [ ]:
#@title 2) 安装系统依赖（ffmpeg + 字体）
!apt-get update -y
!apt-get install -y ffmpeg fonts-noto-cjk libsndfile1
!ffmpeg -version | head -n 2


In [ ]:
#@title 3) 安装 Python 依赖
# 说明：Colab 自带 CUDA 版 torch，避免从 requirements.txt 里重复安装 torch（否则可能被换成CPU版/或耗时升级）
!python -m pip install -U pip
!grep -vE '^torch' requirements.txt > /tmp/requirements_no_torch.txt
!python -m pip install -r /tmp/requirements_no_torch.txt

# 可选：人声分离（Demucs）用于嘈杂/背景音乐场景
!python -m pip install demucs

# torchaudio 新版本保存音频需要 torchcodec（否则 Demucs 会报错）
!python -m pip install torchcodec


In [ ]:
#@title 4) 生成 config.ini（GPU + 中文 + 不润色）
from pathlib import Path

config_text = """[API]
deepseek_api_key =

[Service]
host = 127.0.0.1
port = 50515

[Models]
# 如果显存不够/加载太慢，可改：asr_model_size = small
asr_model_size = medium

# T4 显存有限，默认使用 600M（更稳）；追求质量可改为 1.3B
translation_model = facebook/nllb-200-distilled-600M

use_gpu = true
beam_size = 3

[GPU]
# 在 GPU 重任务前清理 CUDA 缓存，降低 OOM 概率（略慢）
clear_cuda_cache_before_tasks = true

[Translation]
default_target_language = zh
use_deepseek_polish = false

[Audio]
# 人声分离（Demucs）：改善背景音乐/嘈杂场景识别
enable_vocal_separation = true
vocal_separation_model = htdemucs
# 默认用 cuda 加速；如遇 OOM 可改为 cpu
vocal_separation_device = cuda
"""

Path("config.ini").write_text(config_text, encoding="utf-8")
print("wrote config.ini")


In [ ]:
#@title 5) 后台启动服务（不占用单元格）
import os
import pathlib
import signal
import subprocess
import sys
import time

import requests

SERVICE_URL = "http://127.0.0.1:50515"
FORCE_RESTART = False  # True=重启服务并生成 server.log（用于排查 500）

def health():
    try:
        return requests.get(f"{SERVICE_URL}/health", timeout=2).json()
    except Exception:
        return None

def stop_running_server():
    pid_path = pathlib.Path("server.pid")
    if pid_path.exists():
        try:
            pid = int(pid_path.read_text().strip())
            os.kill(pid, signal.SIGTERM)
            print("killed by server.pid:", pid)
        except Exception as e:
            print("failed to kill by server.pid:", e)

    # 兜底：按端口杀（Colab/Linux）
    subprocess.run(["bash", "-lc", "fuser -k 50515/tcp || true"], check=False)

h = health()
if h and FORCE_RESTART:
    print("force restart: stopping existing server ...")
    stop_running_server()
    time.sleep(2)
    h = health()

if not h:
    p = subprocess.Popen(
        [sys.executable, "server_optimized.py"],
        stdout=open("server.log", "wb"),
        stderr=subprocess.STDOUT,
        start_new_session=True,
    )
    pathlib.Path("server.pid").write_text(str(p.pid))
    print("server started, pid:", p.pid)
else:
    print("server already running:", h)
    if not pathlib.Path("server.log").exists():
        print("NOTE: 未找到 server.log；如需抓取服务端报错，请将 FORCE_RESTART=True 再运行本单元格。")


In [ ]:
#@title 6) 等待模型加载完成（首次运行需要等待）
import json
import os
import time
from pathlib import Path

import requests

SERVICE_URL = "http://127.0.0.1:50515"
pid_path = Path("server.pid")
log_path = Path("server.log")

def pid_alive(pid: int) -> bool:
    try:
        os.kill(pid, 0)
        return True
    except Exception:
        return False

def tail_log(max_lines: int = 120) -> str:
    if not log_path.exists():
        return "(server.log not found)"
    try:
        lines = log_path.read_text(errors="ignore").splitlines()
        return "\n".join(lines[-max_lines:])
    except Exception as e:
        return f"(failed to read server.log: {e})"

for i in range(1800):  # 最多等 3600 秒
    try:
        h = requests.get(f"{SERVICE_URL}/health", timeout=5).json()
    except Exception as e:
        if i % 5 == 0:
            print(f"{i*2:>4}s", "waiting for server...", repr(e))
        if pid_path.exists():
            try:
                pid = int(pid_path.read_text().strip() or "0")
            except Exception:
                pid = 0
            if pid and not pid_alive(pid):
                print("\nserver process exited; last logs:\n")
                print(tail_log())
                raise
        time.sleep(2)
        continue

    if h.get("ready"):
        print("READY:\n", json.dumps(h, ensure_ascii=False, indent=2))
        break
    if i % 5 == 0:
        print(f"{i*2:>4}s", h.get("phase"), h.get("progress"), h.get("message"))
    if h.get("phase") == "error":
        raise RuntimeError(h.get("error") or "模型加载失败，请查看 server.log")
    time.sleep(2)
else:
    raise TimeoutError("等待服务就绪超时：请查看 server.log")


In [ ]:
#@title 7) 下载测试视频（自动保存为 1.mp4）
# 直接把你的直链贴到这里即可
VIDEO_URL = "https://oplist.mae5.com/d/本地存储/1.mp4?sign=b58L3c5JYGAMNwLcO09asS9CV8aHTlGBXiO3Yi8Pe0Y=:0"
OUT_PATH = "1.mp4"
FORCE = False  # True=总是重新下载

import urllib.parse
from pathlib import Path

import requests

try:
    from tqdm.auto import tqdm
except Exception:
    tqdm = None

def normalize_url(url: str) -> str:
    parts = urllib.parse.urlsplit(url)
    # 对 path 做编码，避免包含中文路径时部分工具报错
    path = urllib.parse.quote(parts.path)
    return urllib.parse.urlunsplit((parts.scheme, parts.netloc, path, parts.query, parts.fragment))

out = Path(OUT_PATH)
if out.exists() and out.stat().st_size > 0 and not FORCE:
    print("exists:", out, out.stat().st_size)
else:
    url = normalize_url(VIDEO_URL)
    with requests.get(url, stream=True, timeout=60) as r:
        r.raise_for_status()
        total = int(r.headers.get("content-length") or 0)
        if tqdm and total > 0:
            pbar = tqdm(total=total, unit="B", unit_scale=True, desc="download")
        else:
            pbar = None
        with open(out, "wb") as f:
            for chunk in r.iter_content(chunk_size=1024 * 1024):
                if not chunk:
                    continue
                f.write(chunk)
                if pbar:
                    pbar.update(len(chunk))
        if pbar:
            pbar.close()
    print("downloaded:", out, out.stat().st_size)


In [ ]:
#@title 8) 翻译生成中文字幕（不润色）
!python batch_translate.py 1.mp4 -t zh --translation-only


In [ ]:
#@title 9) 合并字幕与视频（生成带字幕 MP4）
import subprocess
from pathlib import Path

VIDEO_PATH = Path("1.mp4")
SRT_PATH = Path("1_zh.srt")
MODE = "hard"  # hard=硬字幕(烧录)；soft=软字幕(可开关)

if not VIDEO_PATH.exists():
    raise FileNotFoundError(VIDEO_PATH)
if not SRT_PATH.exists():
    raise FileNotFoundError(SRT_PATH)

OUT_PATH = Path("1_zh_subbed.mp4")

if MODE == "soft":
    cmd = [
        "ffmpeg",
        "-y",
        "-i",
        str(VIDEO_PATH),
        "-i",
        str(SRT_PATH),
        "-map",
        "0:v",
        "-map",
        "0:a?",
        "-map",
        "1:0",
        "-c:v",
        "copy",
        "-c:a",
        "copy",
        "-c:s",
        "mov_text",
        str(OUT_PATH),
    ]
else:
    encoders = subprocess.check_output(
        ["ffmpeg", "-hide_banner", "-encoders"],
        text=True,
        stderr=subprocess.STDOUT,
    )
    use_nvenc = "h264_nvenc" in encoders

    vf = "subtitles=1_zh.srt:charenc=UTF-8:force_style='FontName=Noto Sans CJK SC,FontSize=24,Outline=1,Shadow=1'"
    cmd = [
        "ffmpeg",
        "-y",
        "-i",
        str(VIDEO_PATH),
        "-vf",
        vf,
        "-c:a",
        "copy",
    ]
    if use_nvenc:
        cmd += ["-c:v", "h264_nvenc", "-preset", "p4", "-cq", "19"]
    else:
        cmd += ["-c:v", "libx264", "-preset", "fast", "-crf", "19"]
    cmd.append(str(OUT_PATH))

print("Running:\n ", " ".join(cmd))
subprocess.run(cmd, check=True)
print("OK:", OUT_PATH)


In [ ]:
#@title 10) 下载生成的字幕 / 带字幕视频
from google.colab import files

files.download("1_zh.srt")
files.download("1_zh_subbed.mp4")


In [ ]:
#@title 11) （可选）停止服务
import os
import signal
import pathlib

pid_path = pathlib.Path("server.pid")
if pid_path.exists():
    pid = int(pid_path.read_text().strip())
    os.kill(pid, signal.SIGTERM)
    print("killed:", pid)
else:
    print("server.pid not found")
